In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

ModuleNotFoundError: No module named 'openai'

In [ ]:
load_dotenv()
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= os.getenv("AUTH")
)
        

In [ ]:
res = client.responses.create(
    model = "deepseek/deepseek-chat-v3.1:free",
    reasoning = { "effort" : "low" },
    store = True,
    instructions = "Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вина.",
    input = "Привет! Какое вино посоветуете?"
)

printx(res.output_text)